In [10]:
# This is necessary to recognize the modules
import os
import sys
import json
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [4]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=False)

In [5]:
from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path, database_name="optimization_database")
optimizer.get_all_study_names()

2024-09-12 14:55:03,414 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16bcb6a40>
2024-09-12 14:55:03,415 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16bf33c70>


['pmm_simple_with_optimizer',
 'macd_bb_v1_with_optimizer',
 'grid_strike_1',
 'grid_strike_2']

In [6]:
df = optimizer.get_study_trials_df("grid_strike_2")
df

,number,value,datetime_start,datetime_complete,duration,params_grid_lower_price_quantile,params_grid_step,params_grid_upper_price_quantile,params_trading_pair,accuracy,...,profit_factor,sharpe_ratio,total_executors,total_executors_with_position,total_long,total_positions,total_short,total_volume,win_signals,state
0,0,0.00311473,2024-09-12 14:22:50.749167,2024-09-12 14:22:52.882544,0 days 00:00:02.133377,0.25,0.02,0.95,DOGS-USDT,0.6,...,1.94951422,1.97556223,48,10,8,10,2,848.94470806,6,COMPLETE
1,1,0.0090071,2024-09-12 14:22:52.890955,2024-09-12 14:22:54.828764,0 days 00:00:01.937809,0.1,0.005,0.9,DOGS-USDT,0.86486486,...,3.45498138,3.74452277,172,74,64,74,10,1990.97627136,64,COMPLETE
2,2,0.01303866,2024-09-12 14:22:54.833791,2024-09-12 14:22:57.272525,0 days 00:00:02.438734,0.25,0.005,0.9,PEOPLE-USDT,1,...,1,4.10820234,216,76,12,76,64,1850.0035836,76,COMPLETE
3,3,0.01653198,2024-09-12 14:22:57.278639,2024-09-12 14:22:57.971084,0 days 00:00:00.692445,0.25,0.015,0.85,PEOPLE-USDT,1,...,1,6.36014175,60,17,3,17,14,1378.25701425,17,COMPLETE
4,4,0.05148438,2024-09-12 14:22:57.978805,2024-09-12 14:22:58.924965,0 days 00:00:00.946160,0.2,0.015,0.8,WIF-USDT,1,...,1,5.85564069,38,20,6,20,14,3793.4244666,20,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1005,0.02786819,2024-09-12 14:29:45.898139,2024-09-12 14:29:46.153086,0 days 00:00:00.254947,0.25,0.015,0.8,1000SATS-USDT,1,...,1,9.54935109,26,14,6,14,8,3406.46537481,14,COMPLETE
1006,1006,0.06252497,2024-09-12 14:29:46.160108,2024-09-12 14:29:46.460889,0 days 00:00:00.300781,0.25,0.015,0.8,WIF-USDT,1,...,1,5.30316522,36,20,6,20,14,4269.67010348,20,COMPLETE
1007,1007,0.05587068,2024-09-12 14:29:46.467483,2024-09-12 14:29:46.823227,0 days 00:00:00.355744,0.25,0.01,0.8,WIF-USDT,1,...,1,6.27838504,50,28,6,28,22,4338.67943487,28,COMPLETE
1008,1008,0.00730804,2024-09-12 14:29:46.831188,2024-09-12 14:29:47.097090,0 days 00:00:00.265902,0.25,0.015,0.8,PEPE-USDT,0.4,...,2.75627721,2.44132505,24,10,10,10,0,2118.66531165,4,COMPLETE


In [14]:
config_id = 946

config_to_backtest = json.loads(df[df["number"] == config_id].to_dict(orient="records")[0]["config"])
config_to_backtest

{'id': 'C385eyWFUL8E3UYCRTow7AFaMsuuyjfdjT498AnVutua',
 'controller_name': 'grid_strike',
 'controller_type': 'generic',
 'total_amount_quote': 1000,
 'manual_kill_switch': None,
 'candles_config': [],
 'connector_name': 'binance',
 'trading_pair': 'WIF-USDT',
 'leverage': 1,
 'position_mode': 'HEDGE',
 'grid_mid_price': 1.552,
 'grid_upper_price': 1.661,
 'grid_lower_price': 1.49825,
 'sniper_upper_price': 2.016,
 'sniper_lower_price': 1.327,
 'grid_allocation': 0.85,
 'inventory_buffer': 0.05,
 'n_levels': 11,
 'grid_step': 0.01,
 'take_profit': None,
 'activation_bounds': None,
 'rebalanced': True}

In [15]:
from research_notebooks.grid_strike.grid_strike_bt import GridStrikeBT

import datetime

start = int(datetime.datetime(2024, 9, 1).timestamp())
end = int(datetime.datetime(2024, 9, 12).timestamp())

config = backtesting.get_controller_config_instance_from_dict(config_to_backtest)
# Running the backtesting this will output a backtesting result object that has built in methods to visualize the results
backtester = GridStrikeBT()
backtesting_result = await backtesting.run_backtesting(
    config=config,
    start=start,
    end=end,
    backtesting_resolution="3m",
    trade_cost=0,
    backtester=backtester,
)
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

2024-09-12 15:08:52,923 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x309bd59c0>
2024-09-12 15:08:52,923 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x30a0c3dc0>, 295180.365952625)]']
connector: <aiohttp.connector.TCPConnector object at 0x309bd7a90>
2024-09-12 15:08:53,691 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x3093a6b60>
2024-09-12 15:08:53,691 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x309aef2b0>
